# Compute SPEI for Brazil and Colombia using weather panel data

In [3]:
path_w = "C:/Users/canzolbr/Documents/GitHub/Forecasting-World-Coffee-Prices/Data/Weather/"

In [4]:
import pandas as pd
# load data set
df_br = pd.read_csv(path_w   + "br_weather.csv")
df_co = pd.read_csv(path_w + "co_weather.csv")

df_br['country'] = 'Brazil'
df_co['country'] = 'Colombia'

In [5]:
# merge states centroids
df_br_c = pd.read_csv(path_w + "google-earth/Brazil/brazil_states_centroids.csv")
df_co_c = pd.read_csv(path_w + "google-earth/Colombia/colombia_states_centroids.csv")

# keep relevant information
df_br_c = df_br_c[['lat','lon','state_code','state_name']] 
df_co_c = df_co_c[['lat','lon','state_code','state_name']] 

In [6]:
df_br = pd.merge(df_br,df_br_c,how='left',on=('state_code','state_name'))
df_co = pd.merge(df_co,df_co_c,how='left',on=('state_code','state_name'))

In [7]:
df = pd.concat([df_br,df_co],axis=0)

In [8]:
df.head(5)

,date,month,state_code,state_name,year,yyyymm,.geo,precip_mm,t2m_c,country,lat,lon
0,1981-01-01,1,665,Acre,1981,198101,"{""type"":""MultiPoint"",""coordinates"":[]}",297.482481,25.058103,Brazil,-9.310338,-70.456557
1,1981-01-01,1,666,Alagoas,1981,198101,"{""type"":""MultiPoint"",""coordinates"":[]}",63.857384,25.771527,Brazil,-9.514498,-36.622864
2,1981-01-01,1,667,Amapa,1981,198101,"{""type"":""MultiPoint"",""coordinates"":[]}",328.858237,24.276920,Brazil,1.450546,-51.975800
3,1981-01-01,1,668,Amazonas,1981,198101,"{""type"":""MultiPoint"",""coordinates"":[]}",283.651789,25.445684,Brazil,-4.186395,-64.692613
4,1981-01-01,1,669,Bahia,1981,198101,"{""type"":""MultiPoint"",""coordinates"":[]}",95.691111,24.056852,Brazil,-12.464797,-41.734031


In [1]:
# compute SPEI
!pip install xclim 

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   --- ---------------------------

In [11]:
import xarray as xr
import xclim

df = df.copy()

df["time"] = pd.to_datetime(
    dict(year=df.year, month=df.month, day=1)
)

ds = (
    df
    .set_index(["state_code", "time"])
    .sort_index()
    .to_xarray()
)

precip = ds["precip_mm"]
tmean  = ds["t2m_c"]
lat    = ds["lat"]

In [14]:
# compute pet
pet = xclim.indices.potential_evapotranspiration(
    tas=tmean,
    method="Hargreaves",
   # freq="MS"
)

KeyError: 'units'

In [ ]:
spei_3  = xclim.indices.spei(
    precipitation=precip,
    pet=pet,
    freq="MS",
    window=3
)

spei_6  = xclim.indices.spei(
    precipitation=precip,
    pet=pet,
    freq="MS",
    window=6
)

spei_12 = xclim.indices.spei(
    precipitation=precip,
    pet=pet,
    freq="MS",
    window=12
)


df_spei = xr.Dataset(
    {
        "spei_3": spei_3,
        "spei_6": spei_6,
        "spei_12": spei_12,
    }
).to_dataframe().reset_index()